# Program description

- This program merges electoral data with worker level data and classifies each candidate as STEM or Non-STEM

# Setting the ambience

In [146]:
import pandas as pd
import re
import pyodbc
import time
import numpy as np
import warnings
from unidecode import unidecode
warnings.simplefilter("ignore") # disable warnings

create_electoral_data_dir = "C:/Users/GabrielCaserDosPasso/Documents/RAIS/1_create_electoral_data/output/data/"
work_dir                  = "C:/Users/GabrielCaserDosPasso/Documents/RAIS/2_create_occupation_data/"
output_dir                = "C:/Users/GabrielCaserDosPasso/Documents/RAIS/2_create_occupation_data/output/"

# Oppening dataset with most voted candidates

In [147]:
df_candidates = pd.read_csv(create_electoral_data_dir + "df_candidatos_2016_clean.csv", sep = ";", dtype = str)

# Deffining STEM professions

In [149]:
cbos_stem = "1223 1236 1237 1413 1425 1426 2011 2012 2021 2030 2031 2032 2033 2034 2111 2112 2122 2123 2124 2131 2132 2133 2134 2140 2142 2143 2144 2145 2146 2147 2148 2149 2211 2212 2221 2222 2341 2342 2343 2344 2513 3001 3003 3011 3012 3111 3112 3115 3121 3122 3123 3161 3171 3172 3180 3181 3182 3183 3185 3186 3187 3201 3212 3253 3951"

- source: Machado et all 2021

# Deffining years and states of interest

In [150]:
anos =[2003]
for i in range(2016 - anos[0]): # mudar depois
    anos.append(anos[-1] + 1)

In [151]:
estados = list(df_candidates.sigla_uf.unique())
#estados = ["SP"]

# Creating list of IDs to search in RAIS

In [152]:
cpfs_rdd = tuple(df_candidates.cpf.unique())

In [153]:
len(cpfs_rdd)

12931

# Login into RAIS server

In [154]:
server = 'sql-sigilo-eastus-prod.71d5dd223a40.database.windows.net'

database = 'RAIS'

port= '1433'

username = 'GabrielCP3'

password = '@VN1ai2GG3as4'

driver= '{SQL Server}'

cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT='+port+';DATABASE='+database+';UID='+username+';PWD='+ password)

cursor = cnxn.cursor()

## Creating function that downloads RAIS data

In [155]:
df = pd.DataFrame()

In [156]:
def rais(ano,estado):
    global df
   
    if ano >= 2011:
        consulta_1 = """ SELECT
                    a.CPF cpf,
                    a.[PIS] pis,
                    a.[Nome Trabalhador] nome,
                    a.[Município] id_municipio,
                    a.[CNPJ Raiz] cnpj_raiz,
                    a.[CBO Ocupação 2002] cbo_2002,
                    a.[Vl Remun Dezembro Nom] rem_dez,
                    a.[Data Admissão Declarada] data_adm,
                    a.[Tempo Emprego] tenure,
                    a.[Vínculo Ativo 31 12] vinc_ativ3112,
                    a.[Qtd Hora Contr] hours
                    FROM dbo.{}{}ID a
                    WHERE a.[Vínculo Ativo 31 12] != '0'
                    AND a.[Vínculo Ativo 31 12] != '0'
                    AND a.[Vínculo Ativo 31 12] != ' 0'
                    AND a.[Vínculo Ativo 31 12] != 0
                    AND a.[CPF] IN {}
                   """.format(estado,ano,cpfs_rdd)
        
        df_provisorio = pd.read_sql(consulta_1,  cnxn)
        
        df_provisorio["Ano"] = ano
        df_provisorio["Estado"] = estado
        
        df_provisorio['cpf'] = df_provisorio['cpf'].str.strip()
    
        df_provisorio.loc[(df_provisorio['vinc_ativ3112'] == ' 1') |
                      (df_provisorio['vinc_ativ3112'] == '1')  |
                      (df_provisorio['vinc_ativ3112'] == 'True'), 'vinc_ativ3112'] = 1
    
        df_provisorio.loc[(df_provisorio['vinc_ativ3112'] == ' 0') |
                      (df_provisorio['vinc_ativ3112'] == '0')  |
                      (df_provisorio['vinc_ativ3112'] == 'False'),'vinc_ativ3112'] = 0
           
        df = pd.concat([df, df_provisorio])
           
        print(f"foi criado um df de tamanho {len(df)}")

    elif ano >= 2006:
        
        consulta_1 = """ SELECT
                        a.CPF cpf,
                        a.[PIS] pis,
                        a.[NOME] nome,
                        a.[MUNICIPIO] id_municipio,
                        a.[RADIC CNPJ] cnpj_raiz,
                        a.[OCUP 2002] cbo_2002,
                        a.[REM DEZEMBRO] rem_dez,
                        a.[DT ADMISSAO] data_adm,
                        a.[TEMP EMPR] tenure,
                        a.[EMP EM 31 12] vinc_ativ3112,
                        a.[HORAS CONTR] hours
                        FROM dbo.{}{}ID a
                        WHERE a.[EMP EM 31 12] != '0'
                        AND a.[EMP EM 31 12] != '0'
                        AND a.[EMP EM 31 12] != ' 0'
                        AND a.[EMP EM 31 12] != 0
                        AND a.[CPF] IN {}
                        """.format(estado,ano,cpfs_rdd)
        
        df_provisorio = pd.read_sql(consulta_1,  cnxn)
        
        df_provisorio["Ano"] = ano
        df_provisorio["Estado"] = estado
        
        df_provisorio['cpf'] = df_provisorio['cpf'].str.strip()

    
        df_provisorio.loc[(df_provisorio['vinc_ativ3112'] == ' 1') |
                      (df_provisorio['vinc_ativ3112'] == '1')  |
                      (df_provisorio['vinc_ativ3112'] == 'True'), 'vinc_ativ3112'] = 1
    
        df_provisorio.loc[(df_provisorio['vinc_ativ3112'] == ' 0') |
                      (df_provisorio['vinc_ativ3112'] == '0')  |
                      (df_provisorio['vinc_ativ3112'] == 'False'),'vinc_ativ3112'] = 0
           
        df = pd.concat([df, df_provisorio])
           
        print(f"foi criado um df de tamanho {len(df)}")
        
        
    else: # 2003 until 2005
        
        consulta_1 = """ SELECT
                        a.CPF cpf,
                        a.[PIS] pis,
                        a.[NOME] nome,
                        a.[MUNICIPIO] id_municipio,
                        a.[RADIC CNPJ] cnpj_raiz,
                        a.[OCUP 2002] cbo_2002,
                        a.[REM DEZEMBRO] rem_dez,
                        a.[DT ADMISSAO] data_adm,
                        a.[TEMP EMPR] tenure,
                        a.[EMP EM 31 12] vinc_ativ3112,
                        a.[HORAS CONTR] hours
                        FROM dbo.{}{}ID a
                        WHERE a.[EMP EM 31 12] != '0'
                        AND a.[EMP EM 31 12] != '0'
                        AND a.[EMP EM 31 12] != ' 0'
                        AND a.[EMP EM 31 12] != 0
                        AND a.[CPF] IN {}
                        """.format(estado,ano,cpfs_rdd)
        df_provisorio = pd.read_sql(consulta_1,  cnxn)
        
        df_provisorio["Ano"] = ano
        df_provisorio["Estado"] = estado
        
        df_provisorio['cpf'] = df_provisorio['cpf'].str.strip()

    
        df_provisorio.loc[(df_provisorio['vinc_ativ3112'] == ' 1') |
                      (df_provisorio['vinc_ativ3112'] == '1')  |
                      (df_provisorio['vinc_ativ3112'] == 'True'), 'vinc_ativ3112'] = 1
    
        df_provisorio.loc[(df_provisorio['vinc_ativ3112'] == ' 0') |
                      (df_provisorio['vinc_ativ3112'] == '0')  |
                      (df_provisorio['vinc_ativ3112'] == 'False'),'vinc_ativ3112'] = 0
           
        df = pd.concat([df, df_provisorio])
           
        print(f"foi criado um df de tamanho {len(df)}")

In [157]:
start_time = time.time()

for x in estados:
    for i in anos:
        rais(i,x)
    
print("My program took", time.time() - start_time, "to run")

foi criado um df de tamanho 8
foi criado um df de tamanho 76
foi criado um df de tamanho 176
foi criado um df de tamanho 278
foi criado um df de tamanho 377
foi criado um df de tamanho 454
foi criado um df de tamanho 564
foi criado um df de tamanho 669
foi criado um df de tamanho 779
foi criado um df de tamanho 872
foi criado um df de tamanho 984
foi criado um df de tamanho 1075
foi criado um df de tamanho 1167
foi criado um df de tamanho 1235
foi criado um df de tamanho 1329
foi criado um df de tamanho 1784
foi criado um df de tamanho 2356
foi criado um df de tamanho 2892
foi criado um df de tamanho 3427
foi criado um df de tamanho 4077
foi criado um df de tamanho 4863
foi criado um df de tamanho 5603
foi criado um df de tamanho 6323
foi criado um df de tamanho 6911
foi criado um df de tamanho 7558
foi criado um df de tamanho 8142
foi criado um df de tamanho 8691
foi criado um df de tamanho 9094
foi criado um df de tamanho 9118
foi criado um df de tamanho 9267
foi criado um df de tama

foi criado um df de tamanho 53794
foi criado um df de tamanho 53958
foi criado um df de tamanho 54097
foi criado um df de tamanho 54237
foi criado um df de tamanho 54358
foi criado um df de tamanho 54470
foi criado um df de tamanho 54580
foi criado um df de tamanho 54695
foi criado um df de tamanho 54775
foi criado um df de tamanho 54782
foi criado um df de tamanho 54838
foi criado um df de tamanho 54902
foi criado um df de tamanho 54962
foi criado um df de tamanho 55025
foi criado um df de tamanho 55100
foi criado um df de tamanho 55187
foi criado um df de tamanho 55286
foi criado um df de tamanho 55373
foi criado um df de tamanho 55450
foi criado um df de tamanho 55525
foi criado um df de tamanho 55585
foi criado um df de tamanho 55643
foi criado um df de tamanho 55688
foi criado um df de tamanho 55736
foi criado um df de tamanho 55933
foi criado um df de tamanho 56174
foi criado um df de tamanho 56435
foi criado um df de tamanho 56688
foi criado um df de tamanho 56962
foi criado um 

# Cleaning RAIS data

In [158]:
df = df.drop(columns = "rem_dez")

In [159]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64772 entries, 0 to 37
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   cpf            64772 non-null  object
 1   pis            64772 non-null  object
 2   nome           64772 non-null  object
 3   id_municipio   64772 non-null  object
 4   cnpj_raiz      64772 non-null  object
 5   cbo_2002       64772 non-null  object
 6   data_adm       64772 non-null  object
 7   tenure         64772 non-null  object
 8   vinc_ativ3112  64772 non-null  object
 9   hours          64772 non-null  object
 10  Ano            64772 non-null  int64 
 11  Estado         64772 non-null  object
dtypes: int64(1), object(11)
memory usage: 6.4+ MB


In [160]:
df = df.drop_duplicates()

In [161]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64740 entries, 0 to 37
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   cpf            64740 non-null  object
 1   pis            64740 non-null  object
 2   nome           64740 non-null  object
 3   id_municipio   64740 non-null  object
 4   cnpj_raiz      64740 non-null  object
 5   cbo_2002       64740 non-null  object
 6   data_adm       64740 non-null  object
 7   tenure         64740 non-null  object
 8   vinc_ativ3112  64740 non-null  object
 9   hours          64740 non-null  object
 10  Ano            64740 non-null  int64 
 11  Estado         64740 non-null  object
dtypes: int64(1), object(11)
memory usage: 6.4+ MB


In [162]:
# keeping only numbers in the cbo_2002 variable
df['cbo_2002_original'] = df['cbo_2002']
df['cbo_2002'] = df['cbo_2002'].str.extract('(\d+)').astype(str)

In [163]:
df["pis"] = df["pis"].str.strip() #removing leading and ending spaces

In [164]:
df["data_adm"] = df["data_adm"].str.strip() #removing leading and ending spaces
df["cbo_2002"] = df["cbo_2002"].str.strip() #removing leading and ending spaces

# Creating variable "tenure" that measures how many months someone worked in a STEM occupation

In [165]:
lista = list(df["tenure"])

In [166]:
# replacing "," for "."
coluna = []

for i in lista:
    if ',' in str(i).lower():  
        coluna.append(i.replace(',', '.'))
    else:
        coluna.append(i)
        
df["tenure"] = coluna

In [167]:
# changing variable types
df["tenure"] = df['tenure'].astype(float)
#df["tenure"] = df['tenure'].astype(int)
#df['data_adm'] = df['data_adm'].astype(int)


In [168]:
df['cnpj_raiz'] = df['cnpj_raiz'].astype(str).str.zfill(8) # filling cnpj with leading 0 until it reaches 8 characters
df['cbo_2002'] = df['cbo_2002'].astype(str).str.zfill(6) # filling cbo with leading 0 until it reaches 6 characters
df['data_adm'] = df['data_adm'].astype(str).str.zfill(8) # filling cbo with leading 0 until it reaches 8 characters

df['data_adm'] = pd.to_datetime(df['data_adm'], format = '%d%m%Y') # changing do date time type

## Keeping CPFs with more than one PIS only for those who match names with TCE dataset

In [169]:
df["n_pis"] = df.groupby('cpf')['pis'].transform('nunique') # number of unique PIS per cpf

In [170]:
# cleaning names
df["nome"] = df["nome"].str.strip() #removing leading and ending spaces
df["nome"] = df["nome"].apply(unidecode) # removing accents
df["nome"] = df["nome"].str.lower() #using lower letters
df["first_name"] = df["nome"].str.split().str[0] #creating first name variable
df["last_name"] = df["nome"].str.split().str[-1] #creating last name variable

# cleaning names from TSE dataset
df_candidates["nome"] = df_candidates["nome"].str.strip() #removing leading and ending spaces
df_candidates["nome"] = df_candidates["nome"].apply(unidecode) #removing accents
df_candidates["nome"] = df_candidates["nome"].str.lower() #using lower letters
df_candidates["first_name"] = df_candidates["nome"].str.split().str[0] #creating first name variable
df_candidates["last_name"] = df_candidates["nome"].str.split().str[-1] #creating last name variable

In [171]:
df = pd.merge(df, df_candidates[['cpf','nome','first_name','last_name','id_municipio']], how = "outer", on = "cpf", indicator = True, validate = "many_to_one", suffixes = ('_rais', '_tse') , )

In [172]:
df._merge.value_counts()

_merge
both          64740
right_only     4379
left_only         0
Name: count, dtype: int64

- 4036 of 13585 candidates did not appeared in RAIS

In [173]:
df = df[df["_merge"] != "right_only"] # dropping TSE candidates that were not in RAIS

In [174]:
df['equal_names'] = (df["first_name_rais"] == df["first_name_tse"]) # creating dummy that indicates wether the name in RAIS is the same as the name in TSE 

In [175]:
df = df[(df['n_pis'] == 1) | df['equal_names'] == True] # keeping only observations in RAIS which have only one PIS or have the same first_name in RAIS and TSE datasets 

In [176]:
df = df.drop(columns = ['equal_names', '_merge', 'id_municipio_tse', 'last_name_tse', 'first_name_tse', 'nome_tse', 'last_name_rais', 'first_name_rais'])

In [177]:
df["n_pis_new"] = df.groupby('cpf')['pis'].transform('nunique')

In [178]:
df.describe()

,data_adm,tenure,Ano,n_pis,n_pis_new
count,64264,64264.000000,64264.000000,64264.000000,64264.000000
mean,2001-05-08 23:56:26.231793792,115.263410,2009.971135,1.132983,1.113734
min,1959-11-06 00:00:00,0.000000,2003.000000,1.000000,1.000000
25%,1994-08-08 18:00:00,22.900000,2007.000000,1.000000,1.000000
50%,2004-04-01 00:00:00,70.000000,2010.000000,1.000000,1.000000
75%,2009-01-01 00:00:00,188.900000,2013.000000,1.000000,1.000000
max,2016-12-31 00:00:00,589.800000,2016.000000,18.000000,4.000000
std,NaN,113.516959,3.538090,0.457243,0.355153


- there are still a few CPFs with multiple PIS. But now they seem to be the same person because they have the same name

## Creating identificator of contracts per person

In [179]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64264 entries, 0 to 64739
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   cpf                64264 non-null  object        
 1   pis                64264 non-null  object        
 2   nome_rais          64264 non-null  object        
 3   id_municipio_rais  64264 non-null  object        
 4   cnpj_raiz          64264 non-null  object        
 5   cbo_2002           64264 non-null  object        
 6   data_adm           64264 non-null  datetime64[ns]
 7   tenure             64264 non-null  float64       
 8   vinc_ativ3112      64264 non-null  object        
 9   hours              64264 non-null  object        
 10  Ano                64264 non-null  float64       
 11  Estado             64264 non-null  object        
 12  cbo_2002_original  64264 non-null  object        
 13  n_pis              64264 non-null  float64       
 14  n_pis_new  

In [180]:
df["count"] = df.groupby(['cpf','data_adm',"cbo_2002"])["cbo_2002"].transform('count') # this variable indicates that person p is working in a contract c

In [181]:
df["contract_id"] = df.groupby(['cpf','data_adm',"cbo_2002","cnpj_raiz"]).ngroup() # this variable indicates that person p is working in a contract c

In [182]:
df["n_contracts"] = df.groupby('cpf')['contract_id'].transform('nunique') # this variable indicates the number of different contracts that each person had
df["n_contracts_year"] = df.groupby(['cpf','Ano'])['contract_id'].transform('nunique') # the same but per year

In [183]:
df.describe()

,data_adm,tenure,Ano,n_pis,n_pis_new,count,contract_id,n_contracts,n_contracts_year
count,64264,64264.000000,64264.000000,64264.000000,64264.000000,64264.000000,64264.000000,64264.000000,64264.000000
mean,2001-05-08 23:56:26.231793792,115.263410,2009.971135,1.132983,1.113734,5.232976,12075.003750,4.249378,1.441320
min,1959-11-06 00:00:00,0.000000,2003.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000
25%,1994-08-08 18:00:00,22.900000,2007.000000,1.000000,1.000000,2.000000,6202.750000,2.000000,1.000000
50%,2004-04-01 00:00:00,70.000000,2010.000000,1.000000,1.000000,4.000000,12109.000000,3.000000,1.000000
75%,2009-01-01 00:00:00,188.900000,2013.000000,1.000000,1.000000,8.000000,17891.000000,5.000000,2.000000
max,2016-12-31 00:00:00,589.800000,2016.000000,18.000000,4.000000,23.000000,24012.000000,31.000000,9.000000
std,NaN,113.516959,3.538090,0.457243,0.355153,3.690494,6822.330247,3.309209,0.786488


## Creating "contract_tenure" 

- it calculates the total time that a worker was in a specific contract

In [184]:
df["contract_first_year"] = df.groupby(['contract_id'])["Ano"].transform('min') # first year a contract appears in Rais
df["contract_last_year"] = df.groupby(['contract_id'])["Ano"].transform('max') # last year a contract appears in Rais

In [185]:
df["contract_first_year_tenure"] = df[(df["contract_first_year"] == df["Ano"])]["tenure"] # getting the inital value of tenure per contract
df["contract_last_year_tenure"] = df[(df["contract_last_year"] == df["Ano"])]["tenure"] # getting the final value of tenure per contract

In [186]:
df["contract_first_year_tenure"] = df.groupby(['contract_id'])["contract_first_year_tenure"].transform('sum') # coping this value to all the observations of that contract
df["contract_last_year_tenure"] = df.groupby(['contract_id'])["contract_last_year_tenure"].transform('sum') # coping this value to all the observations of that contract

In [187]:
df["contract_tenure"] = df["contract_last_year_tenure"] - df["contract_first_year_tenure"]

In [188]:
df.loc[(df["contract_first_year"] == df["contract_last_year"]) & (df["contract_first_year_tenure"] <= 12), "contract_tenure"] = df["contract_first_year_tenure"] # using the first years' tenure all the total value if the contract only has one year and the tenure in the first yera is less or equal to 12
df.loc[(df["contract_first_year"] == df["contract_last_year"]) & (df["contract_first_year_tenure"] > 12), "contract_tenure"] = 12 # replacing the total duration of a contract for 12 when the contract only appeared one year and the tenure of the first contract was bigger than 12. 

- The last replacement was necessary, because we do not observe before 2003. Therefore, since tenure does not reset when a worker change its' occupation inside the same company, a worker could have been working in a non-STEM job for 120 months, but only in the last 12 months he worked in a STEM job. Therefore, we would be superestimating his STEM experience. When we change this value to 12, it means that we're only considering his experience from 2003 until the yera of his election 

- Negatives in min value for contract_tenure is strange, but I'll ignore it for now since it does not happen for STEM occupations

# Classifing occupations as STEM

In [190]:
df["stem_job"] = df["cbo_2002"].apply(lambda x: 1 if x[0:4] in cbos_stem else 0) # using the list from Machado(2021) to classify each occupation as a stem_job

## Keeping only last year of each contract

In [191]:
last_year_contract = df.groupby(['contract_id'])['Ano'].idxmax()# creating a dataset that keeps only the last year of a contract c from person p 
df = df.loc[last_year_contract]

## Saving

In [192]:
df.to_csv(output_dir + 'data/mayors_jobs_2016.csv', sep = ",", index = False, encoding = "utf-8-sig")

# Keeping only STEM contracts

In [193]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24013 entries, 1269 to 18288
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   cpf                         24013 non-null  object        
 1   pis                         24013 non-null  object        
 2   nome_rais                   24013 non-null  object        
 3   id_municipio_rais           24013 non-null  object        
 4   cnpj_raiz                   24013 non-null  object        
 5   cbo_2002                    24013 non-null  object        
 6   data_adm                    24013 non-null  datetime64[ns]
 7   tenure                      24013 non-null  float64       
 8   vinc_ativ3112               24013 non-null  object        
 9   hours                       24013 non-null  object        
 10  Ano                         24013 non-null  float64       
 11  Estado                      24013 non-null  object      

In [194]:
df = df.query("stem_job == 1")

In [195]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 650 entries, 43685 to 3691
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   cpf                         650 non-null    object        
 1   pis                         650 non-null    object        
 2   nome_rais                   650 non-null    object        
 3   id_municipio_rais           650 non-null    object        
 4   cnpj_raiz                   650 non-null    object        
 5   cbo_2002                    650 non-null    object        
 6   data_adm                    650 non-null    datetime64[ns]
 7   tenure                      650 non-null    float64       
 8   vinc_ativ3112               650 non-null    object        
 9   hours                       650 non-null    object        
 10  Ano                         650 non-null    float64       
 11  Estado                      650 non-null    object        

## Keeping only one contract per year

In [196]:
df = df.groupby(['cpf']).apply(lambda x: x.loc[x['contract_tenure'].idxmax()]) # for person p at year y keeps the contract with the biggest tenure
df = df.reset_index(drop = True)

In [197]:
df['hours'] = df['hours'].astype(float)

In [198]:
# in case there are contracts with the same number of tenure, keep the one with biggest hours contracted
df = df.groupby(['cpf']).apply(lambda x: x.loc[x['hours'].idxmax()]) # for person p at year y keeps the contract with the biggest hours
df = df.reset_index(drop = True)

In [199]:
# In case person p at year y has more than one contract with the same number of tenure and hours, we keep one of them ramdomly
df = df.groupby("cpf").apply(lambda x: x.sample(1)) # group by ID and select a random row from each ID
df.reset_index(drop = True, inplace = True)

- Duplicates were happening because we tried to keep only the occupation with the highest number of months that a person worked, there are some people that worked the same amount in different occupations. Therefore we'll keep one of the duplicated choosing it randomly  

In [200]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 436 entries, 0 to 435
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   cpf                         436 non-null    object        
 1   pis                         436 non-null    object        
 2   nome_rais                   436 non-null    object        
 3   id_municipio_rais           436 non-null    object        
 4   cnpj_raiz                   436 non-null    object        
 5   cbo_2002                    436 non-null    object        
 6   data_adm                    436 non-null    datetime64[ns]
 7   tenure                      436 non-null    float64       
 8   vinc_ativ3112               436 non-null    object        
 9   hours                       436 non-null    float64       
 10  Ano                         436 non-null    float64       
 11  Estado                      436 non-null    object        

In [201]:
df.describe()

,data_adm,tenure,hours,Ano,n_pis,n_pis_new,count,contract_id,n_contracts,n_contracts_year,contract_first_year,contract_last_year,contract_first_year_tenure,contract_last_year_tenure,contract_tenure,stem_job
count,436,436.000000,436.000000,436.000000,436.000000,436.000000,436.000000,436.000000,436.000000,436.000000,436.000000,436.000000,436.000000,436.000000,436.000000,436.0
mean,1999-09-02 07:58:53.944954112,141.659174,35.392202,2010.490826,1.133028,1.100917,3.653670,11949.295872,4.229358,1.279817,2007.559633,2010.490826,106.452523,141.659174,38.726147,1.0
min,1970-05-12 00:00:00,0.700000,1.000000,2003.000000,1.000000,1.000000,1.000000,44.000000,1.000000,1.000000,2003.000000,2003.000000,0.400000,0.700000,0.700000,1.0
25%,1990-05-03 06:00:00,26.900000,30.000000,2006.000000,1.000000,1.000000,1.000000,6021.500000,2.000000,1.000000,2004.000000,2006.000000,10.900000,26.900000,12.000000,1.0
50%,2002-05-01 12:00:00,101.999998,40.000000,2010.000000,1.000000,1.000000,2.000000,11667.500000,4.000000,1.000000,2007.000000,2010.000000,47.800000,101.999998,12.000000,1.0
75%,2008-12-08 18:00:00,226.825000,44.000000,2015.000000,1.000000,1.000000,5.000000,17917.750000,6.000000,1.000000,2010.000000,2015.000000,191.400000,226.825000,48.000002,1.0
max,2015-12-17 00:00:00,487.899994,44.000000,2016.000000,6.000000,3.000000,13.000000,23988.000000,18.000000,4.000000,2016.000000,2016.000000,451.900000,487.899994,144.100012,1.0
std,NaN,128.036228,10.218074,4.218995,0.429607,0.330655,3.236539,6905.105597,2.815827,0.590854,3.634272,4.218995,113.117778,128.036228,41.580013,0.0


## Creating column "cbo_max" that indicates the STEM job with more tenure

In [202]:
df["total_tenure"] = df.groupby(['cpf'])['contract_tenure'].transform('sum') # total months in STEM occupations per person 

In [203]:
df.describe()

,data_adm,tenure,hours,Ano,n_pis,n_pis_new,count,contract_id,n_contracts,n_contracts_year,contract_first_year,contract_last_year,contract_first_year_tenure,contract_last_year_tenure,contract_tenure,stem_job,total_tenure
count,436,436.000000,436.000000,436.000000,436.000000,436.000000,436.000000,436.000000,436.000000,436.000000,436.000000,436.000000,436.000000,436.000000,436.000000,436.0,436.000000
mean,1999-09-02 07:58:53.944954112,141.659174,35.392202,2010.490826,1.133028,1.100917,3.653670,11949.295872,4.229358,1.279817,2007.559633,2010.490826,106.452523,141.659174,38.726147,1.0,38.726147
min,1970-05-12 00:00:00,0.700000,1.000000,2003.000000,1.000000,1.000000,1.000000,44.000000,1.000000,1.000000,2003.000000,2003.000000,0.400000,0.700000,0.700000,1.0,0.700000
25%,1990-05-03 06:00:00,26.900000,30.000000,2006.000000,1.000000,1.000000,1.000000,6021.500000,2.000000,1.000000,2004.000000,2006.000000,10.900000,26.900000,12.000000,1.0,12.000000
50%,2002-05-01 12:00:00,101.999998,40.000000,2010.000000,1.000000,1.000000,2.000000,11667.500000,4.000000,1.000000,2007.000000,2010.000000,47.800000,101.999998,12.000000,1.0,12.000000
75%,2008-12-08 18:00:00,226.825000,44.000000,2015.000000,1.000000,1.000000,5.000000,17917.750000,6.000000,1.000000,2010.000000,2015.000000,191.400000,226.825000,48.000002,1.0,48.000002
max,2015-12-17 00:00:00,487.899994,44.000000,2016.000000,6.000000,3.000000,13.000000,23988.000000,18.000000,4.000000,2016.000000,2016.000000,451.900000,487.899994,144.100012,1.0,144.100012
std,NaN,128.036228,10.218074,4.218995,0.429607,0.330655,3.236539,6905.105597,2.815827,0.590854,3.634272,4.218995,113.117778,128.036228,41.580013,0.0,41.580013


In [204]:
df = df.rename(columns = {"tenure": "tenure_rais"})

In [205]:
df = df.rename(columns = {"total_tenure": "tenure"})

In [206]:
df["max_contract_tenure"] = df.groupby(['cpf'])['contract_tenure'].transform('max') # creates a variable with the highest number of months that a person worked 

In [207]:
df = df[(df["max_contract_tenure"] == df["contract_tenure"])] # keeping only the occupation with the highest number of months that a person worked  

- é possível que as duplicatas estejam vindo daqui. Se um cara trabalhou o mesmo tempo em duas profissões, vai dar duplicado.
- uma alternativa é manter apenas a profissão com o salário mais alto no caso de duplicata.

In [209]:
# droping acessory variables
df = df.drop(columns = ['Ano', 'contract_first_year_tenure', 'n_contracts_year', 'contract_last_year_tenure', 'contract_first_year','contract_last_year', 'contract_tenure', 'max_contract_tenure', 'contract_id', 'count', 'n_pis', 'cbo_2002_original', 'cnpj_raiz'])

In [210]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 436 entries, 0 to 435
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   cpf                436 non-null    object        
 1   pis                436 non-null    object        
 2   nome_rais          436 non-null    object        
 3   id_municipio_rais  436 non-null    object        
 4   cbo_2002           436 non-null    object        
 5   data_adm           436 non-null    datetime64[ns]
 6   tenure_rais        436 non-null    float64       
 7   vinc_ativ3112      436 non-null    object        
 8   hours              436 non-null    float64       
 9   Estado             436 non-null    object        
 10  n_pis_new          436 non-null    int64         
 11  n_contracts        436 non-null    int64         
 12  stem_job           436 non-null    int64         
 13  tenure             436 non-null    float64       
dtypes: datetim

## Merging electoral data (df_canidates) with STEM mayors' occupational data (df_tenure_only_stem) 

In [211]:
df = pd.merge(df, df_candidates, how = "outer", on = "cpf", indicator = True, validate = "one_to_one")

In [212]:
df["stem_job"].fillna(0, inplace=True) # classifies all candidates that didn't have a formal STEM job as a Non-STEM candidate

In [213]:
df["tenure"].fillna(0, inplace = True) # gives 0 of exposure intensity to Non-STEM candidates

In [214]:
df.describe()

,data_adm,tenure_rais,hours,n_pis_new,n_contracts,stem_job,tenure
count,436,436.000000,436.000000,436.000000,436.000000,12931.000000,12931.000000
mean,1999-09-02 07:58:53.944954112,141.659174,35.392202,1.100917,4.229358,0.033717,1.305746
min,1970-05-12 00:00:00,0.700000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,1990-05-03 06:00:00,26.900000,30.000000,1.000000,2.000000,0.000000,0.000000
50%,2002-05-01 12:00:00,101.999998,40.000000,1.000000,4.000000,0.000000,0.000000
75%,2008-12-08 18:00:00,226.825000,44.000000,1.000000,6.000000,0.000000,0.000000
max,2015-12-17 00:00:00,487.899994,44.000000,3.000000,18.000000,1.000000,144.100012
std,NaN,128.036228,10.218074,0.330655,2.815827,0.180508,10.345534


# Creating variable "rdd_stem_won" that indicates whether the STEM candidates won 

In [215]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12931 entries, 0 to 12930
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   cpf                12931 non-null  object        
 1   pis                436 non-null    object        
 2   nome_rais          436 non-null    object        
 3   id_municipio_rais  436 non-null    object        
 4   cbo_2002           436 non-null    object        
 5   data_adm           436 non-null    datetime64[ns]
 6   tenure_rais        436 non-null    float64       
 7   vinc_ativ3112      436 non-null    object        
 8   hours              436 non-null    float64       
 9   Estado             436 non-null    object        
 10  n_pis_new          436 non-null    float64       
 11  n_contracts        436 non-null    float64       
 12  stem_job           12931 non-null  float64       
 13  tenure             12931 non-null  float64       
 14  ano   

In [216]:
teste = pd.DataFrame(df.groupby(["id_municipio"]).stem_job.sum())

In [217]:
teste.rename(columns = {"stem_job": "n_stem_candidates"}, inplace = True)

In [218]:
df = df.drop(columns = "_merge")

In [219]:
df = pd.merge(df, teste, how = "left", on = "id_municipio", indicator = True)

In [220]:
df.n_stem_candidates.value_counts()

n_stem_candidates
0.0    11846
1.0     1031
2.0       54
Name: count, dtype: int64

In [221]:
df.query("resultado == 'eleito' & stem_job == 1").shape

(167, 38)

In [222]:
df["stem_won"] = ((df["resultado"] == 'eleito') & (df["stem_job"] == 1)).astype(int) # creates dummy

# Droping stem candidates whose election was invalidated

In [223]:
df[df["stem_job"] == 1].situacao.value_counts()

situacao
deferido                420
deferido com recurso     16
Name: count, dtype: int64

# Saving

In [224]:
# droping variables that we won't use
df = df.drop(columns = ['_merge', 'data_adm'])

In [225]:
# changing types
df.ano = df.ano.astype(int)
df.idade = df.idade.astype(float)
df.dif_votos_2_lugar = df.dif_votos_2_lugar.astype(float)
df.dif_votos_3_lugar = df.dif_votos_3_lugar.astype(float)

In [226]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12931 entries, 0 to 12930
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   cpf                12931 non-null  object 
 1   pis                436 non-null    object 
 2   nome_rais          436 non-null    object 
 3   id_municipio_rais  436 non-null    object 
 4   cbo_2002           436 non-null    object 
 5   tenure_rais        436 non-null    float64
 6   vinc_ativ3112      436 non-null    object 
 7   hours              436 non-null    float64
 8   Estado             436 non-null    object 
 9   n_pis_new          436 non-null    float64
 10  n_contracts        436 non-null    float64
 11  stem_job           12931 non-null  float64
 12  tenure             12931 non-null  float64
 13  ano                12931 non-null  int32  
 14  tipo_eleicao       12931 non-null  object 
 15  sigla_uf           12931 non-null  object 
 16  id_municipio       129

- cbo_2002 has a lower number of observations because we only got information on STEM occupations
- dif_votos_3_lugar has a lower number of obs because we consider it missing when municipalities had less than 3 candidates

In [227]:
df.describe()

,tenure_rais,hours,n_pis_new,n_contracts,stem_job,tenure,ano,idade,dif_votos_2_lugar,dif_votos_3_lugar,n_stem_candidates,stem_won
count,436.000000,436.000000,436.000000,436.000000,12931.000000,12931.000000,12931.0,12930.000000,12931.000000,7543.000000,12931.000000,12931.000000
mean,141.659174,35.392202,1.100917,4.229358,0.033717,1.305746,2016.0,49.359319,0.161358,0.388245,0.088083,0.012915
std,128.036228,10.218074,0.330655,2.815827,0.180508,10.345534,0.0,10.789145,0.156098,0.170052,0.297797,0.112911
min,0.700000,1.000000,1.000000,1.000000,0.000000,0.000000,2016.0,21.000000,0.000000,0.003633,0.000000,0.000000
25%,26.900000,30.000000,1.000000,2.000000,0.000000,0.000000,2016.0,42.000000,0.051898,0.256850,0.000000,0.000000
50%,101.999998,40.000000,1.000000,4.000000,0.000000,0.000000,2016.0,49.000000,0.117414,0.407434,0.000000,0.000000
75%,226.825000,44.000000,1.000000,6.000000,0.000000,0.000000,2016.0,57.000000,0.219736,0.509563,0.000000,0.000000
max,487.899994,44.000000,3.000000,18.000000,1.000000,144.100012,2016.0,90.000000,0.981252,0.981894,2.000000,1.000000


In [228]:
df.to_csv(output_dir + 'data/base_rdd_covid_stem_2016.csv', sep = ",", index = False, encoding = "utf-8-sig")

In [229]:
teste = df[df['stem_job'] == 1]

In [230]:
import seaborn as sns

In [231]:
teste.tenure.value_counts()

tenure
12.000000     155
24.000000      32
36.000000      19
12.000000      12
84.000000       9
             ... 
36.100000       1
7.900000        1
108.100000      1
84.000003       1
0.700000        1
Name: count, Length: 120, dtype: int64